# Пример парсинга страницы сайта

## Requests

Для того, чтобы получить html-код страницы нам потребуется библиотека requests:

In [3]:
import requests

req = requests.get('http://zadolba.li/20160417')

In [4]:
print req

<Response [200]>


In [5]:
print type(req)

<class 'requests.models.Response'>


In [1]:
# print req.text

## Beautiful Soup

Теперь нам нужно как-то обрабатывать этот html-код. Для этого подойдет библиотека Beautiful Soup 4:

In [7]:
import bs4

У bs4 весьма несложный интерфейс, хотя обращаться к документации на первых порах все же придется.

In [8]:
parser = bs4.BeautifulSoup(req.text, 'lxml')

In [1]:
# print type(parser)
# print parser

Выделим первый тег div, атрибут class у которого имеет значение 'text':

In [2]:
# print parser.find('div', attrs={'class':'text'})

In [11]:
x = parser.find('div', attrs={'class':'text'})
print type(x)

<class 'bs4.element.Tag'>


In [3]:
# print x.text

Выделим тексты всех историй со страницы:

In [13]:
y = parser.findAll('div', attrs={'class':'text'})
print type(y)

<class 'bs4.element.ResultSet'>


In [4]:
# for result in y:
#     print result.text
#     print "\n------\n"

## Multiprocessing

Уже рассмотренных простых действий достаточно для того, чтобы кое-как парсить сайт с известной вам структурой. Но если вы попробуете таким образом распарсить более одной страницы, скорее всего заметите, что это происходит очень медленно. Можно существенно ускориться, воспользовавшись библиотекой multiprocessing, чтобы параллельно парсить несколько страниц. Ниже приводится пример такого кода:

In [ ]:
%%writefile parse_zadolbali.py
import requests
import bs4
from multiprocessing import Pool
import codecs

def parse_page(url):
    text = requests.get(url).text
    parser = bs4.BeautifulSoup(text, 'lxml')
    x = parser.findAll('div', attrs={'class':'text'})
    return [res.text for res in x]

p = Pool(10)
url_list = ['http://zadolba.li/201604' + '0' * int(n < 10) + str(n) for n in range(1, 18)]
    
if __name__ == '__main__':    
    map_results = p.map(parse_page, url_list)
    reduce_results = reduce(lambda x,y: x + y, map_results)
    with codecs.open('parsing_results.txt', 'w', 'utf-8') as output_file:
        print >> output_file, u'\n'.join(reduce_results)

parse_zadolbali.py надо запускать из консоли, а не из блокнота - ipython notebook не слишком дружит с multiprocessing

На практике прирост скорости парсинга наблюдается при увеличение пула примерно до 100, далее уже бессмысленно, но все-таки не стоит слишком усердствовать - 10-20 будет достаточно.

## Scrapy

Владельцы сайта, который вы парсите, могут не очень хотеть, чтобы вы это делали. Тем более, если вы делаете это очень активно. Возможно периодически вас будут на некоторое время банить. Вы можете написать парсер так, чтобы он проверял ответ на запрос и после нескольких неудачных попыток засыпал на сколько-то секунд. Можно пытаться делать вид, что ваш парсер вовсе не парсер, делая запросы через случайные интервалы времени. Но беспокоиться о повторных запросах, скорости работы и других нюансах не обязательно самому. Существуют питоновские библиотеки, специально предназначенные для парсинга сайтов.

Одна из таких библиотек - Scrapy. При выполнении задания на следующей неделе вы можете попробовать воспользоваться ей. Возможно у вас будут некоторые сложности при установке и в процессе привыкания к css selectors, но в будущем умение использовать scrapy или другую готовую библиотеку для парсинга сайтов почти наверняка оправдает себя.